In [ ]:
# !pip install tensorflow-gpu
# !pip install keras
# !pip install keras-bert

In [1]:
import numpy as np
from keras_bert import load_trained_model_from_checkpoint
import tokenization
import re

class BertContextAugmentation():
    #TODO try catch
    def __init__(self, model_folder):
        self.folder = model_folder
        self.config_path = folder+'/bert_config.json'
        self.checkpoint_path = folder+'/bert_model.ckpt'
        self.vocab_path = folder+'/vocab.txt'
        self.tokenizer = tokenization.FullTokenizer(vocab_file=self.vocab_path, do_lower_case=False)
        self.model = load_trained_model_from_checkpoint(self.config_path, self.checkpoint_path, training=True)
        
        
    def bert_aug(self, cls_sentence):
        # предсказание слов, закрытых токеном MASK в фразе. На вход нейросети надо подать фразу в формате: [CLS] Я пришел в [MASK] и купил [MASK]. [SEP]

        # входная фраза с закрытыми словами с помощью [MASK]
        #sentence = 'Я пришел в [MASK] и купил [MASK].'  #@param {type:"string"}
        out_sentence = cls_sentence
        sentence = cls_sentence

        # преобразование в токены (tokenizer.tokenize() не обрабатывает [CLS], [MASK], поэтому добавим их вручную)
        sentence = sentence.replace(' [MASK] ','[MASK]'); sentence = sentence.replace('[MASK] ','[MASK]'); sentence = sentence.replace(' [MASK]','[MASK]')  # удаляем лишние пробелы. Можно заменить регуляркой "\s?\[MASK\]\s?", но это надо импортить re
        sentence = sentence.split('[MASK]')             # разбиваем строку по маске
        tokens = ['[CLS]']                              # фраза всегда должна начинаться на [CLS]
        # обычные строки преобразуем в токены с помощью tokenizer.tokenize(), вставляя между ними [MASK]
        for i in range(len(sentence)):
            if i == 0:
                tokens = tokens + self.tokenizer.tokenize(sentence[i]) 
            else:
                tokens = tokens + ['[MASK]'] + self.tokenizer.tokenize(sentence[i]) 
        tokens = tokens + ['[SEP]']                     # фраза всегда должна заканчиваться на [SEP] 
        # в tokens теперь токены, которые гарантированно по словарю преобразуются в индексы
        
        # преобразуем в массив индексов, который можно подавать на вход сети, причем число 103 в нем это [MASK]
        token_input = self.tokenizer.convert_tokens_to_ids(tokens)        
        # удлиняем до 512 длины
        token_input = token_input + [0] * (512 - len(token_input))
        
        
        # создаем маску, заменив все числа 103 на 1, а остальное 0
        mask_input = [0]*512
        for i in range(len(mask_input)):
            if token_input[i] == 103:
                mask_input[i] = 1
        #print(mask_input)
        
        # маска фраз (вторая фраза маскируется числом 1, а все остальное числом 0)
        seg_input = [0]*512
        
        
        # конвертируем в numpy в форму (1,) -> (1,512)
        token_input = np.asarray([token_input])
        mask_input = np.asarray([mask_input])
        seg_input = np.asarray([seg_input])
        
        
        # пропускаем через нейросеть...
        predicts = self.model.predict([token_input, seg_input, mask_input])[0]       # в [0] полная фраза с заполненными предсказанными словами на месте [MASK]
        predicts = np.argmax(predicts, axis=-1)
        
        
        # форматируем результат в строку, разделенную пробелами
        predicts = predicts[0][:len(tokens)]    # длиной как исходная фраза (чтобы отсечь случайные выбросы среди нулей дальше)
        out = []
        # добавляем в out только слова в позиции [MASK], которые маскированы цифрой 1 в mask_input
        for i in range(len(mask_input[0])):
            if mask_input[0][i] == 1:           # [0][i], т.к. требование было (1,512)
                out.append(predicts[i]) 
        
        out = self.tokenizer.convert_ids_to_tokens(out)# индексы в токены
        
        for i in range(len(re.findall('\[MASK\]?', out_sentence))):
            out_sentence = re.sub('\[MASK\]', out[i], out_sentence, 1)

        return out_sentence
    
    def choose_random_place(self, splited_sentence, sent_length):
        """
        Выбирает рандомные места в предложении куда в дальнейшем вставляет слово
        """
        aug_num = np.random.randint(1, sent_length // 3 + 1)
        for i in range(aug_num):
            splited_sentence = splited_sentence[:]; splited_sentence.insert(np.random.randint(1, sent_length),'[MASK]')
        return ' '.join(splited_sentence)
            
        
    def choose_random_word(self, splited_sentence, sent_length):
        """
        Выберает рандомное слово которое будет заменено
        """
        aug_num = np.random.randint(1, sent_length // 3 + 1)
        for i in range(aug_num):
            rand_ind = np.random.randint(0, sent_length)
            for n, i in enumerate(splited_sentence):
                if n == rand_ind:
                    splited_sentence[n] = '[MASK]'
        return ' '.join(splited_sentence)    
        
    def split_for_aug(self, sent_length, n):
        if sent_length <= n:
            return False
        return True
        
    def split_sentence(self, splited_sentence, n):
        """
        Yield successive n-sized chunks from lst.
        """
        for i in range(0, len(splited_sentence), n):
            yield splited_sentence[i:i + n]
    
    
    def make_one_part_aug(self, splited_sentence, sent_length, attempts, trys):
        if sent_length // 3 > 0:
            aug_sentence = self.try_another_one_aug(splited_sentence, sent_length, attempts=attempts, trys=trys)
            return aug_sentence
        return ' '.join(splited_sentence)
    
    def make_single_aug(self, sentence, splited_sentence, sent_length):
        """
        Применяет одну из двух аугментаций к предложению
        """
        if np.random.randint(0, 2) == 0:
            aug_sentence = self.bert_aug(self.choose_random_place(splited_sentence, sent_length))
        else:
            aug_sentence = self.bert_aug(self.choose_random_word(splited_sentence, sent_length))
        return aug_sentence
    
    def try_another_one_aug(self, sentence, splited_sentence, sent_length, attempts=1, trys=3):
        """
        Пытается применить аугментацию пока не получит новое предложение.
        """
        if attempts <= trys:
            aug_sentence = self.make_single_aug(sentence, splited_sentence, sent_length)
            if aug_sentence == sentence:
                self.try_another_one_aug(sentence, splited_sentence, sent_length, attempts=attempts+1, trys=trys)
            else:
                return aug_sentence
        return sentence

    def make_aug(self, sentence, attempts=1, trys=3):
        splited_sentence = re.findall(r"[\w']+|[^\w\s]", sentence)
        sent_length = len(splited_sentence)
        if self.split_for_aug(sent_length, 200):
            aug_sentence = ''
            for splited_sentence_part in self.split_sentence(splited_sentence, 200):
                aug_sentence += self.make_one_part_aug(' '.join(splited_sentence_part), splited_sentence_part, len(splited_sentence_part), attempts, trys)
            return aug_sentence
        aug_sentence = self.make_one_part_aug(sentence, splited_sentence, sent_length, attempts, trys)
        return aug_sentence
        
        
    def make_one_part_aug(self, sentence, splited_sentence, sent_length, attempts, trys):
        if sent_length // 3 > 0:
            aug_sentence = self.try_another_one_aug(sentence, splited_sentence, sent_length, attempts=attempts, trys=trys)
            return aug_sentence
        return sentence

Using TensorFlow backend.
C:\Users\Ilya\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ilya\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ilya\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ilya\Anaconda3

In [2]:
folder = 'models/ru_conversational_cased_L-12_H-768_A-12'
bert = BertContextAugmentation(folder)

In [ ]:
folder = 'model'
download_url = 'http://files.deeppavlov.ai/deeppavlov_data/bert/ru_conversational_cased_L-12_H-768_A-12.tar.gz'

# print('Downloading model')
# zip_path = '{}.zip'.format(folder)
# !test -d $folder || (wget $download_url && unzip $zip_path)

In [4]:
cls_sentence = 'Бразильскиебиологи обнаружили, что самцы пауковвида Manogea porracea заботятсяо потомстве: вместе с самкой они защищаюткладку яиц и атакуют хищников,приближающихся к паутине. Больше того,частосамцы ухаживают за кладкой в одиночку.По мнению ученых, это первый известныйвид одиночных пауков, в котором самецзаботится о потомстве наравне с самкой.Исследование опубликованов Animal Behaviour.Умногих видов пауков, после того, каксамка откладывает яйца, она в одиночкузаботится о кладке и, иногда, об ужевылупившихся паучатах. Самки охраняюткладки в паутине или гнезде, обеспечиваютпаучат пищей. Паучихи некоторых видов(например, Agelenalabyrinthica)умирают в период инкубации яиц, а детенышипервое время после вылупления поедаюттруп матери. Пауки-самцы практическиникогда не принимают участия в уходеза потомством. Исключением стали социальныепауки Stegodyphusdumicola, которые плетут общую ловчую сеть и обитают вблизи друг от друга. Некоторые самцы этого вида отгоняют от кладкимуравьев, пытающихся ее украсть. Темудивительнее стало для авторовисследования поведение самцов одиночныхпауков M.porracea.Пауки M. porracea обитают в ЮжнойАмерике, в ареале от Панамы до Аргентины.Самка плетет трехмерную паутину,состоящую из ловчей сети и куполообразнойструктуры, в которой она проводит большуючасть времени. Во время брачного периодасамец делает над паутиной самки свою,такой же формы и связывает ее с паутинойсамки поперечными нитями. Послеоплодотворения самка откладывает яйца,делает из них несколько коконов изпаутины и подвешивает кладки к поперечнымнитям, соединяющим родительские паутины.Затем, вплоть до вылупления паучат, одинили оба родителя охраняют кладки. Ониследят за тем, чтобы они не упала наземлю и в случае необходимости«ремонтируют» паутину, и защищают коконыот хищников (пауков-пиратов, которыепожирают яйца, паучат и взрослых особей).Авторы нового исследования решиливыяснить, как влияет забота и уход самцовна выживаемость их потомства.Ученые исследовали M. porracea, обитающихна эвкалиптовой плантации на юго-востокеБразилии. Они наблюдали за пауками вовремя брачного периода, в ноябре-декабре2013 года. Исследователи сравнивали,сколько паучат вылупилось из кладок закоторыми присматривали либо оба родителя,либо только самец или самка, либо ониоставались бесхозными. Отдельно авторыработы следили за тем, как самцы «штопают»паутину и как это влияет на количествовылупившихся паучат. Также, чтобы оценить выживаемость последних, ученые провелилабораторные эксперименты, во времякоторых они подсаживали на несколькодней пауков-хищников к неохраняемойкладке, либо к кладке, за которойприсматривал самец M. porracea.Оказалось, что, если закладками ухаживали оба родителя илитолько самка, среднее количествовылупившихся из одной кладки паучатбыло практически одинаковым (14 и 13 особейсоответственно). Если за яйцамиприсматривал только самец, выживалопримерно на 30 процентов меньше отпрысков.Если же кладки оставались «беспризорными»,из них, в среднем, вылуплялось на 70процентов меньше паучат, чем из тех, закоторыми ухаживали оба родителя илитолько мать. Также ученые выяснили, что,если кладка оставалась без присмотра, то почти в половине случаев паутина, ккоторой она крепилась, обрывалась, икокон падал на землю. Лабораторныеэксперименты показали, что самцыэффективно защищали кладки от хищныхпауков: выживаемость потомства была на30-40 процентов выше в тех случаях, когдакокон защищал от отец, чем в случаях,когда хищник оставался наедине с кладкой.Интересным результатомнаблюдений оказалось то, что к концубрачного периода число одиноких отцов,охраняющих своих отпрысков, стало почтив пять раз больше количества самок.Ученые объясняют это тем, что пауки-пираты,нападающие на M. porracea, предпочитаютсамок, которые больше самцов по размерами, по-видимому, содержат больше жиров,которые необходимы паукам-хищникам.Унекоторых пауков есть интересныестратегии заботы о потомстве. Так, самкаюжнорусскоготарантула(Lycosa singoriensis)прикрепляеткокон с яйцами к себе на брюшко и ходитс ним до тех пор, пока паучата не начнутшевелиться. После этого самка сбрасываеткокон и разгрызает его, чтобы помочьдетенышам выбраться. На этом забота опотомстве не заканчивается: паучатавзбираются на спину матери и ездят наней до тех пор, пока не начнут питатьсясамостоятельно. У пауков Coelotesterrestris молодьостается в материнском гнезде примерномесяц после выхода из кокона. Самкаприносит им полупереваренную добычу изащищает от пауков других видов, которыхона выгоняет из гнезда или убивает.Екатерина Русакова'
out = bert.make_aug(cls_sentence)

print(out)

Бразильскиебиологи обнаружили , что самцы пауков Manogea porracea заботятсяо потомстве : вместе с яйцами они защищаюткладку яиц и атакуют хищников , приближающихся к паутине . Больше того , частосамцы ухаживают за кладкой в одиночку . По мнению ученых , это первый известныйвид одиночных пауков , в котором самецзаботится о потомстве наравне с самкой . Исследование опубликованов Animal Behaviour . Умногих видов пауков , после того , каксамка откладывает яйца , она в одиночкузаботится о кладке яиц , иногда , об ужевылупившихся паучатах . Самки охраняюткладки в паутине или гнезде , обеспечиваютпаучат ##ели . Паучихи некоторых видов ( например , Agelenalabyrinthica ) умирают в период инкубации яиц , а детенышипервое время после вылупления поедаюттруп матери . Пауки - самцы практическиникогда не принимают участия в уходеза потомством . Исключением стали социальныепауки ##цы , которые имеют общую ловчую сеть и обитают независимо друг от друга . Некоторые самцы этого вида отгоняют от самцов , 

In [ ]:
  def split_for_aug(self, sent_length, n):
        if sent_length <= n:
            return False
        return True
        
    def split_sentence(self, splited_sentence, n):
        """
        Yield successive n-sized chunks from lst.
        """
        for i in range(0, len(splited_sentence), n):
            yield splited_sentence[i:i + n]
    
    
    def make_one_part_aug(self, splited_sentence, sent_length, attempts, trys):
        if sent_length // 3 > 0:
            aug_sentence = self.try_another_one_aug(splited_sentence, sent_length, attempts=attempts, trys=trys)
            return aug_sentence
        return ' '.join(splited_sentence)
    
    
    def make_aug(self, sentence, attempts=1, trys=3):
        splited_sentence = re.findall(r"[\w']+|[^\w\s]", sentence)
        sent_length = len(splited_sentence)
        if self.split_for_aug(sent_length, 200):
            aug_sentence = ''
            for splited_sentence_part in self.split_sentence(splited_sentence, 200):
#                 aug_sentence += ' '.join(self.make_one_part_aug(splited_sentence_part, sent_length, attempts, trys))
                print(self.make_one_part_aug(splited_sentence_part, sent_length, attempts, trys))
            return aug_sentence
        aug_sentence = self.make_one_part_aug(splited_sentence, sent_length, attempts, trys)
        return aug_sentence